# Setup and start offline azimuthal pipeline

This notebook will read an existing azint file and extract the integration settings. 
These settings will be saved in a file: ```azint_config.py``` that can subsequently be edited with your favorite text editor to change the integration parameters.  

The only input required in this notebook is the scan number of an integrated file that will serve as a template for the integration configurations.  

The offline Azint integration pipeline itself is run in a terminal, details on that can be found in the last cell [**last cell**](#instructions) in this notebook.  

In [ ]:
import os
import sys
sys.path.append('../')
import DanMAX as DM
import h5py
import datetime
print('Current proposal and visit:')
print(os.getcwd().split('scripts')[0])

#### Generate integration configurations using a previously integrated scan as template  
Specify the scan numbers of the scans you wish to integrate.  
Specify a template scan number that is used for generating the configuration file. If None, use the first scan in ths scan list.

Pro tip:  
Use  
`scans = [int(DM.getScan_id(scan).split('-')[-1]) for scan in DM.findAllScans(scan_type='timescan')]`  
to get a list of all *timescan* scan numbers in the current proposal.

In [ ]:
# Specification of scans to be integrated [list of int] (do not use zero-padding)
scans = range(firstScan, lastScan+1) # Specified as a range, remeber to end with lastScan+1
#scans = [scan1, scan2, scanN] # Manually specified list - a single scan can be specified, but has to be of type list, i.e. with '[scan]'

template_scan = None

# if no template scan is specified, use the first scan in the list
if template_scan == None:
    template_scan = scans[0]

# Find the file path from the scan number 
# alternatively insert path for the .h5 file - TIP: Use tap for auto-complete
#fname = '/data/visitors/danmax/20230160/2023101208/raw/SAMPLE/scan-XXXX.h5'    
fname = DM.findScan(template_scan)

# get the azimuthally integrated filename from master file name
aname = DM.getAzintFname(fname)

# get the path to the poni file (not supported via DanMAX library)
poni_file = h5py.File(aname)['entry/azint/input/poni_file'].attrs['filename']

# read the integrated data and get number of radial bins
data, meta = DM.getAzintData(aname, get_meta = True)
if type(data['q']) != type(None):
    unit = 'q'
    radial_bins = data.get('q').shape[0]
else:
    unit = '2th'
    radial_bins = data.get('tth').shape[0]

# generate the config file text
myConfig = f"""
# This file was generated on {datetime.datetime.now().strftime('%c')} using
# {aname} 
# as a template

# path to poni file [string]
poni_file = f'{poni_file}'

# path to mask file in npy format [string]
mask_file = f'{meta.get('input').get('mask_file').decode()}'

# settings used in the integration
user_config = {{'poni': poni_file, # do not change this line
               'mask': mask_file, # do not change this line
               'radial_bins': {radial_bins}, # number of radial bins [int]
               'azimuth_bins': {data.get('azi_edge')}, # number of azimuthal bins (or list specifying the azimuthal bins)
               'n_splitting': {meta.get('input').get('n_splitting')}, # pixel splitting - number of subdivisions [int[]]
               'error_model': {meta.get('error_model')}, # Error model: use 'None' or 'poisson' [str]
               'polarization_factor': {meta.get('input').get('polarization_factor')}, # Polarization factor - use 0.999997 [float]
               'unit': '{unit}' # Radial unit: use either 'q' or '2th' [str]
               }} 

# Specification of scans to be integrated [list of int]
scans = {list(scans)}
#scans = range(firstScan, lastScan+1) # Specified as a range, remeber to end with lastScan+1
#scans = [scan1, scan2, scanN] # Manually specified list
"""

# write to file in **/scripts/offlineAzint/azint_config.py
with open('offline_azint_config.py','w') as config_file:
    config_file.write(myConfig.strip())

print(f'The configuration file has been saved in:\n{os.path.join(os.getcwd(),"offline_azint_config.py")}\n')
print('Open this file and edit it to set all integration parameters.')
print('Remeber to edit the scan numbers to be integrated!\n\n')
print('The content of configuration file is show below:')
print(80*'-')
print(myConfig.strip())

## How to use the offline azint integration pipeline <a id='instructions'></a>

The offline integration pipeline can unfortunately not be run in a notebook but has to be run in a terminal via `jupyterhub.maxiv.lu.se`:
Open a terminal using the **+** icon in the upper left corner
In the Launcher page select **Terminal** under **Other**
In the terminal run the following command: `source activate hdf5-kernel-environment` to activate the correct environment
The terminal should now show `(hdf5-kernel-environment)` in front of your user name.

Navigate to the offlineAzint folder (where this notebook is located): `cd /data/visitors/danmax/PROPOSAL/VISIT/scripts/offlineAzint`

In this folder you should have the following files:

<table><tr><th>File <th><th>Description <tr><tr>
<tr><td><tt>startOfflineAzint.ipynb</tt> <td><td> This notebook. <tr><tr>
<tr><td><tt>offline_azint_config.py</tt> <td><td> Configuration file created by this notebook. <br> This file is read by the pipeline. <tr><tr>
<tr><td><tt>offline_azint_config_DEFAULT.py</tt> <td><td> A default configuration file for manual editing <br> Make a cope of this file before editing. <br> The new file must be called <tt>'azint_config.py'</tt> <tr><tr>
<tr><td><tt>start_offline_pipeline_DanMAX.py</tt> <td><td> This python script is used to start the pipeline <br> DO NOT EDIT THIS FILE! <tr><tr>
<tr><td><tt>pipeline.py</tt> <td><td> This is the python script containg the pipeline program <br> DO NOT EDIT THIS FILE! <tr><tr>
<tr><td><tt>danmax-pilatus.yaml</tt> <td><td> A settings file for the Pilatus <br> DO NOT EDIT THIS FILE! <tr><tr>
<table>

After running this notebook your `offline_azint_config.py` has been updated based on the integration parameters in the choosen file. By editing `offline_azint_config.py` you can change the integration parameters.

If you want to set up the integration completely from scratch you can make a copy of `offline_azint_config_DEFAULT.py` to `offline_azint_config.py` and edit it to your preference. In this case: Don't forget to specify which scans you want to integrate!
    
The integration is started by `python start_offline_pipeline_danmax.py` a will show provide progress bars during the integration.
    
#### HPC
It is possible to run the integration using the offline HPC for extra speed. Instructions should be the same as here... 
